In [733]:
import pandas as pd
import mysql.connector
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')


# Datos sucios

In [734]:
patients_batch1 = pd.read_csv("../data/patients_batch1.csv")
patients_batch1

,ehr,birth_date,diagnosis_date,death_date
0,10011773,1959-07-05,2015-04-02,NaN
1,10020495,1953-10-02,2017-12-04,NaN
2,10030299,1966-08-16,2019-06-27,NaN
3,10030824,1953-03-03,2018-09-07,NaN
4,10041592,1959-07-06,2018-11-19,NaN
...,...,...,...,...
7495,77575810,1958-03-21,2014-02-22,NaN
7496,77578551,1948-05-18,2018-07-11,NaN
7497,77579212,1976-02-13,2022-03-22,NaN
7498,77605742,1950-03-16,2015-06-29,NaN


In [735]:
patients_batch2 = pd.read_csv("../data/patients_batch2.csv")
patients_batch2

,ehr,birth_date,diagnosis_date,death_date
0,72992494,13/02/1942,08/05/2018,NaN
1,73002338,22/10/1949,08/09/2018,NaN
2,73008149,04/12/1969,12/04/2018,NaN
3,73012939,18/09/1973,17/04/2014,01/01/2018
4,73019870,07/12/1955,24/05/2018,NaN
...,...,...,...,...
2995,99966287,16/12/1978,02/08/2018,NaN
2996,99981192,10/04/1949,28/10/2017,NaN
2997,99988958,21/01/1966,27/05/2013,NaN
2998,99992350,04/09/1939,17/12/2018,NaN


In [736]:
gynecological = pd.read_excel("../data/gynecological.xlsx")
gynecological

,Unnamed: 0,ehr,pregnancy,birth,caesarean,abort,menarche_age,menopause_age
0,0,10011773,-8.0,0.0,NaN,0.0,NaN,71.0
1,1,10030299,3.0,NaN,0.0,0.0,NaN,NaN
2,2,10030824,0.0,0.0,0.0,NaN,NaN,44.0
3,3,10053435,2.0,NaN,0.0,1.0,21.0,74.0
4,4,10111454,0.0,-6.0,NaN,0.0,16.0,55.0
...,...,...,...,...,...,...,...,...
7681,7681,99948591,9.0,2.0,-4.0,0.0,12.0,47.0
7682,7682,99961100,2.0,2.0,0.0,0.0,19.0,52.0
7683,7683,99981192,3.0,3.0,0.0,-5.0,11.0,NaN
7684,7684,99992350,4.0,3.0,NaN,-6.0,11.0,55.0


In [737]:
histochemistry = pd.read_json("../data/histochemistry.json")
histochemistry

,ehr,er,her2,ki67,pr
0,10011773,0.0,0.0,19.0,NaN
1,10020495,1.0,0.0,9.0,0.0
2,10030299,1.0,NaN,18.0,1.0
3,10030824,1.0,0.0,NaN,1.0
4,10041592,0.0,0.0,65.0,0.0
...,...,...,...,...,...
9995,99966287,1.0,0.0,NaN,0.0
9996,99981192,1.0,0.0,14.0,0.0
9997,99988958,NaN,0.0,67.0,NaN
9998,99992350,0.0,0.0,16.0,0.0


In [738]:
if os.path.exists('../data/tumor.xlsx'):
    tumor = pd.read_excel("../data/tumor.xlsx")
else:
    config = {'host': "localhost",
              'port': 3306,
              'user': "cancer_mama_user",
              'passwd': "cancer_mama_password",
              'db': "cancer_mama"}
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()

    sql_columns_names = "SHOW COLUMNS from tumor;"
    sql_tumor = "SELECT * FROM tumor"

    cursor.execute(sql_tumor)
    tumor = pd.DataFrame(cursor.fetchall())

    cursor.execute(sql_columns_names)
    columns_names = pd.DataFrame(cursor.fetchall())

    cursor.close()
    conn.close()

    tumor = tumor.set_axis(columns_names[0], axis=1)
    tumor.to_excel("../data/tumor.xlsx")
tumor

,Unnamed: 0,ehr,n_tumor,t_category,n_category,m_category,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_diagnosis,stage_after_neo,grade,ductal,lobular,neoadjuvant
0,0,10011773,1,IS,0,0,NaN,NaN,NaN,0,NaN,1,NaN,NaN,no
1,1,10020495,1,1,0,0,NaN,NaN,NaN,IA,NaN,2,1.0,NaN,no
2,2,10020495,2,3,1,0,2,0.0,NaN,IA,NaN,2,1.0,NaN,yes
3,3,10030299,1,1,0,0,NaN,NaN,NaN,IA,NaN,1,1.0,NaN,no
4,4,10030824,1,2,1,0,2,2.0,0.0,IIIA,IIIA,3,NaN,1.0,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11162,11162,99966287,1,1,1,0,1,0.0,0.0,IB,IIA,3,1.0,NaN,yes
11163,11163,99981192,1,1,0,0,NaN,NaN,NaN,IA,NaN,2,NaN,NaN,no
11164,11164,99988958,1,0,2,0,1,1.0,0.0,IIIA,IIA,1,1.0,NaN,yes
11165,11165,99992350,1,2,0,0,3,2.0,0.0,IIA,IIIA,2,1.0,NaN,yes


# Preprocesamiento

## Patients

In [739]:
len(patients_batch1.ehr.unique())

7500

In [740]:
len(patients_batch2.ehr.unique())

3000

Vemos si hay algún paciente común en ambos batchs de datos de pacientes

In [741]:
any(patients_batch1['ehr'].isin(patients_batch2['ehr']))

True

Obtenemos los ids de los pacientes que aparecen en ambos 

In [742]:
elementos_comunes = set(patients_batch1['ehr']).intersection(set(patients_batch2['ehr']))
print(f"El número total de pacientes comunes son: {len(elementos_comunes)}")

El número total de pacientes comunes son: 500


Vemos ahora si estos pacientes que aparecen en ambos documentos tienen valores distintos en cada uno de los documentos

In [743]:
pacientes_comunes_batch1 = patients_batch1[patients_batch1['ehr'].isin(elementos_comunes)]
pacientes_comunes_batch1

,ehr,birth_date,diagnosis_date,death_date
7000,72992494,1942-02-13,2018-05-08,NaN
7001,73002338,1949-10-22,2018-09-08,NaN
7002,73008149,1969-12-04,2018-04-12,NaN
7003,73012939,1973-09-18,2014-04-17,2018-01-01
7004,73019870,1955-12-07,2018-05-24,NaN
...,...,...,...,...
7495,77575810,1958-03-21,2014-02-22,NaN
7496,77578551,1948-05-18,2018-07-11,NaN
7497,77579212,1976-02-13,2022-03-22,NaN
7498,77605742,1950-03-16,2015-06-29,NaN


In [744]:
pacientes_comunes_batch2 = patients_batch2[patients_batch2['ehr'].isin(elementos_comunes)]
pacientes_comunes_batch2

,ehr,birth_date,diagnosis_date,death_date
0,72992494,13/02/1942,08/05/2018,NaN
1,73002338,22/10/1949,08/09/2018,NaN
2,73008149,04/12/1969,12/04/2018,NaN
3,73012939,18/09/1973,17/04/2014,01/01/2018
4,73019870,07/12/1955,24/05/2018,NaN
...,...,...,...,...
495,77575810,21/03/1958,22/02/2014,NaN
496,77578551,18/05/1948,11/07/2018,NaN
497,77579212,13/02/1976,22/03/2022,NaN
498,77605742,16/03/1950,29/06/2015,NaN


Parece ser que los valores sí que son iguales, pero cada documento usa un formato distinto para las fechas. Vamos a ponerlo todo en el formato del primer documento, que es el formato por defecto de datetime: aaaa-mm-dd

In [745]:
pacientes_comunes_batch2['birth_date'] = pd.to_datetime(pacientes_comunes_batch2['birth_date'], format='%d/%m/%Y')
pacientes_comunes_batch2['death_date'] = pd.to_datetime(pacientes_comunes_batch2['death_date'], format='%d/%m/%Y')
pacientes_comunes_batch2['diagnosis_date'] = pd.to_datetime(pacientes_comunes_batch2['diagnosis_date'], format='%d/%m/%Y')

pacientes_comunes_batch1['birth_date'] = pd.to_datetime(pacientes_comunes_batch1['birth_date'])
pacientes_comunes_batch1['diagnosis_date'] = pd.to_datetime(pacientes_comunes_batch1['diagnosis_date'])
pacientes_comunes_batch1['death_date'] = pd.to_datetime(pacientes_comunes_batch1['death_date'])

pacientes_comunes_batch2

,ehr,birth_date,diagnosis_date,death_date
0,72992494,1942-02-13,2018-05-08,NaT
1,73002338,1949-10-22,2018-09-08,NaT
2,73008149,1969-12-04,2018-04-12,NaT
3,73012939,1973-09-18,2014-04-17,2018-01-01
4,73019870,1955-12-07,2018-05-24,NaT
...,...,...,...,...
495,77575810,1958-03-21,2014-02-22,NaT
496,77578551,1948-05-18,2018-07-11,NaT
497,77579212,1976-02-13,2022-03-22,NaT
498,77605742,1950-03-16,2015-06-29,NaT


In [746]:
pacientes_comunes_batch1['birth_date'] = pd.to_datetime(pacientes_comunes_batch1['birth_date'])
pacientes_comunes_batch1['diagnosis_date'] = pd.to_datetime(pacientes_comunes_batch1['diagnosis_date'])
pacientes_comunes_batch1['death_date'] = pd.to_datetime(pacientes_comunes_batch1['death_date'])

In [747]:
pacientes_comunes_batch1.reset_index(drop=True, inplace=True)

In [748]:
pacientes_comunes_batch2.equals(pacientes_comunes_batch1)

True

Vemos que todos los valores de los pacientes que aparecen en los dos documentos son exactamente iguales

In [749]:
patients_batch1

,ehr,birth_date,diagnosis_date,death_date
0,10011773,1959-07-05,2015-04-02,NaN
1,10020495,1953-10-02,2017-12-04,NaN
2,10030299,1966-08-16,2019-06-27,NaN
3,10030824,1953-03-03,2018-09-07,NaN
4,10041592,1959-07-06,2018-11-19,NaN
...,...,...,...,...
7495,77575810,1958-03-21,2014-02-22,NaN
7496,77578551,1948-05-18,2018-07-11,NaN
7497,77579212,1976-02-13,2022-03-22,NaN
7498,77605742,1950-03-16,2015-06-29,NaN


In [750]:
patients_batch2

,ehr,birth_date,diagnosis_date,death_date
0,72992494,13/02/1942,08/05/2018,NaN
1,73002338,22/10/1949,08/09/2018,NaN
2,73008149,04/12/1969,12/04/2018,NaN
3,73012939,18/09/1973,17/04/2014,01/01/2018
4,73019870,07/12/1955,24/05/2018,NaN
...,...,...,...,...
2995,99966287,16/12/1978,02/08/2018,NaN
2996,99981192,10/04/1949,28/10/2017,NaN
2997,99988958,21/01/1966,27/05/2013,NaN
2998,99992350,04/09/1939,17/12/2018,NaN


In [751]:
patients_batch2['birth_date'] = pd.to_datetime(patients_batch2['birth_date'], format='%d/%m/%Y')
patients_batch2['diagnosis_date'] = pd.to_datetime(patients_batch2['diagnosis_date'], format='%d/%m/%Y')
patients_batch2['birth_date'] = pd.to_datetime(patients_batch2['birth_date'], format='%d/%m/%Y')

patients_batch1['birth_date'] = pd.to_datetime(patients_batch1['birth_date'])
patients_batch1['diagnosis_date'] = pd.to_datetime(patients_batch1['diagnosis_date'])
patients_batch1['death_date'] = pd.to_datetime(patients_batch1['death_date'])

In [752]:
patients_batch1_sin_comunes = patients_batch1[~patients_batch1["ehr"].isin(pacientes_comunes_batch1["ehr"])]
patients_batch1_sin_comunes

,ehr,birth_date,diagnosis_date,death_date
0,10011773,1959-07-05,2015-04-02,NaT
1,10020495,1953-10-02,2017-12-04,NaT
2,10030299,1966-08-16,2019-06-27,NaT
3,10030824,1953-03-03,2018-09-07,NaT
4,10041592,1959-07-06,2018-11-19,NaT
...,...,...,...,...
6995,72964107,1959-01-01,2019-12-18,NaT
6996,72975846,1959-07-02,2017-09-01,NaT
6997,72976171,1972-12-01,2018-06-09,NaT
6998,72983987,1962-08-28,2013-07-03,2019-01-25


In [753]:
patients = pd.concat([patients_batch1_sin_comunes, patients_batch2], ignore_index=True)
patients

,ehr,birth_date,diagnosis_date,death_date
0,10011773,1959-07-05,2015-04-02,NaN
1,10020495,1953-10-02,2017-12-04,NaN
2,10030299,1966-08-16,2019-06-27,NaN
3,10030824,1953-03-03,2018-09-07,NaN
4,10041592,1959-07-06,2018-11-19,NaN
...,...,...,...,...
9995,99966287,1978-12-16,2018-08-02,NaN
9996,99981192,1949-04-10,2017-10-28,NaN
9997,99988958,1966-01-21,2013-05-27,NaN
9998,99992350,1939-09-04,2018-12-17,NaN


In [754]:
len(patients.ehr.unique())

10000

Como vemos tan solo hay valores nulos en la variable 'death_date'. Esto sinifica que la mayoria de los pacientes no han muerto. Prodedemos a cambiar esta variable por una binaria que indique tan solo si el paciente ha fallecido o no.

In [755]:
patients.isnull().sum()

ehr                  0
birth_date           0
diagnosis_date       0
death_date        8865
dtype: int64

In [756]:
patients['dead'] = patients['death_date'].notnull().astype(int)
patients.drop(columns='death_date', inplace=True)

In [757]:
patients

,ehr,birth_date,diagnosis_date,dead
0,10011773,1959-07-05,2015-04-02,0
1,10020495,1953-10-02,2017-12-04,0
2,10030299,1966-08-16,2019-06-27,0
3,10030824,1953-03-03,2018-09-07,0
4,10041592,1959-07-06,2018-11-19,0
...,...,...,...,...
9995,99966287,1978-12-16,2018-08-02,0
9996,99981192,1949-04-10,2017-10-28,0
9997,99988958,1966-01-21,2013-05-27,0
9998,99992350,1939-09-04,2018-12-17,0


## Histochemistry

In [758]:
histochemistry

,ehr,er,her2,ki67,pr
0,10011773,0.0,0.0,19.0,NaN
1,10020495,1.0,0.0,9.0,0.0
2,10030299,1.0,NaN,18.0,1.0
3,10030824,1.0,0.0,NaN,1.0
4,10041592,0.0,0.0,65.0,0.0
...,...,...,...,...,...
9995,99966287,1.0,0.0,NaN,0.0
9996,99981192,1.0,0.0,14.0,0.0
9997,99988958,NaN,0.0,67.0,NaN
9998,99992350,0.0,0.0,16.0,0.0


Todos los valores estan dentro de los rangos posibles salvo la variable 'ki67' que al ser un valor porcentual no podria sobrepasar el valor 100, y sin embargo vemos que su valor maximo es 137.

In [759]:
histochemistry.describe()

,ehr,er,her2,ki67,pr
count,1.000000e+04,9042.000000,9006.000000,9032.000000,9018.000000
mean,5.494471e+07,0.662906,0.158894,21.315766,0.665669
std,2.609028e+07,0.472743,0.365598,22.242693,0.471782
min,1.001177e+07,0.000000,0.000000,-57.000000,0.000000
25%,3.214794e+07,0.000000,0.000000,12.000000,0.000000
50%,5.480334e+07,1.000000,0.000000,18.000000,1.000000
75%,7.761323e+07,1.000000,0.000000,28.000000,1.000000
max,9.999442e+07,1.000000,1.000000,137.000000,1.000000


En todas la columnas nos encontramos con valores nulos.

In [760]:
histochemistry.isnull().sum()

ehr       0
er      958
her2    994
ki67    968
pr      982
dtype: int64

No hay ninguna fila completamente vacia

In [761]:
filas_nulas = histochemistry[histochemistry.columns.drop('ehr')].isnull().all(axis=1)
filas_nulas[filas_nulas].index.tolist()

[]

Sin embargo hay 488 filas con 2 o mas valores nulos. Procedemos a eliminarlas puesto que este df tiene tan solo 4 variables y si faltan la mitad o mas consideramos que no aporta informacion sufieciente. Ademas tan solo hay 488 filas a las que les sucede, que corresponden tan solo a un 5% del dataset. 

In [762]:
filas_nulas = histochemistry[histochemistry.columns.drop('ehr')].isnull().sum(axis=1) >=2
len(filas_nulas[filas_nulas].index.tolist())

488

In [763]:
histochemistry.drop(filas_nulas[filas_nulas].index.tolist(), inplace=True)

Sigue habiendo valores nulos, pero al menos ahora sabemos que en cada fila como mucho hay tan solo 1 valor nulo

In [764]:
histochemistry.isnull().sum()

ehr       0
er      699
her2    757
ki67    702
pr      725
dtype: int64

In [765]:
len(histochemistry.ehr.unique())

9512

Coinciden todos los ids con los de patients

In [766]:
elementos_comunes = set(histochemistry['ehr']).intersection(set(patients['ehr']))
len(elementos_comunes)

9512

Como comparten todos los ids simplemente añadimos la informacion del nuevo df al de 'patients'

In [767]:
patients = pd.merge(histochemistry, patients, on='ehr')

In [768]:
patients

,ehr,er,her2,ki67,pr,birth_date,diagnosis_date,dead
0,10011773,0.0,0.0,19.0,NaN,1959-07-05,2015-04-02,0
1,10020495,1.0,0.0,9.0,0.0,1953-10-02,2017-12-04,0
2,10030299,1.0,NaN,18.0,1.0,1966-08-16,2019-06-27,0
3,10030824,1.0,0.0,NaN,1.0,1953-03-03,2018-09-07,0
4,10041592,0.0,0.0,65.0,0.0,1959-07-06,2018-11-19,0
...,...,...,...,...,...,...,...,...
9507,99963879,1.0,0.0,18.0,1.0,1959-10-13,2021-07-20,0
9508,99966287,1.0,0.0,NaN,0.0,1978-12-16,2018-08-02,0
9509,99981192,1.0,0.0,14.0,0.0,1949-04-10,2017-10-28,0
9510,99992350,0.0,0.0,16.0,0.0,1939-09-04,2018-12-17,0


## Gynecological

In [769]:
gynecological

,Unnamed: 0,ehr,pregnancy,birth,caesarean,abort,menarche_age,menopause_age
0,0,10011773,-8.0,0.0,NaN,0.0,NaN,71.0
1,1,10030299,3.0,NaN,0.0,0.0,NaN,NaN
2,2,10030824,0.0,0.0,0.0,NaN,NaN,44.0
3,3,10053435,2.0,NaN,0.0,1.0,21.0,74.0
4,4,10111454,0.0,-6.0,NaN,0.0,16.0,55.0
...,...,...,...,...,...,...,...,...
7681,7681,99948591,9.0,2.0,-4.0,0.0,12.0,47.0
7682,7682,99961100,2.0,2.0,0.0,0.0,19.0,52.0
7683,7683,99981192,3.0,3.0,0.0,-5.0,11.0,NaN
7684,7684,99992350,4.0,3.0,NaN,-6.0,11.0,55.0


In [770]:
gynecological.drop(columns=['Unnamed: 0'], inplace=True)

Observamos como hay valores fuera de rango. Para las variables 'pregnancy', 'birth', 'caesarean', 'abort' los valores minimos son negativos, y los maximos dudamos que puedan ser reales. Para la variable 'menopause_age' el valor maximo es 145, lo cual es imposible. Habra que tratar estos valores asi como deberemos asegurarnos que el valor de pregnancy = birth + caesarean + abort.

In [771]:
gynecological.describe()

,ehr,pregnancy,birth,caesarean,abort,menarche_age,menopause_age
count,7.686000e+03,6413.000000,6417.000000,6131.000000,6235.000000,4901.000000,4486.000000
mean,5.505640e+07,2.078590,1.673991,0.029848,0.259984,12.736584,50.263041
std,2.621591e+07,3.052473,3.006228,2.677790,2.691482,3.062710,17.196300
min,1.001177e+07,-10.000000,-10.000000,-11.000000,-12.000000,0.000000,2.000000
25%,3.202165e+07,1.000000,0.000000,0.000000,0.000000,11.000000,44.000000
50%,5.503108e+07,2.000000,2.000000,0.000000,0.000000,13.000000,49.000000
75%,7.794185e+07,3.000000,3.000000,0.000000,1.000000,14.000000,53.000000
max,9.999442e+07,14.000000,15.000000,12.000000,12.000000,26.000000,145.000000


Hay una gran cantidad de nulos, sobre todo para la variable 'menopause_age', siendo el 41.6% de los valores nulos. Teniendo esto en cuenta procederemos a no considerar esta variable. 

In [772]:
gynecological.isnull().sum()

ehr                 0
pregnancy        1273
birth            1269
caesarean        1555
abort            1451
menarche_age     2785
menopause_age    3200
dtype: int64

In [773]:
gynecological.drop(columns='menopause_age', inplace=True)

Sin contar la variable identificadora 'ehr' tenemos tan solo 5 variables explicativas en este dataset. Porcedemos a eliminar aquellas filas que tengan tan solo 1 valor no nulo. Podriamos optar a eliminar aquellas con tan solo 2 valores no nulos, pero primero debremos estudiar si podemos imputar nosotros esos valores.

In [774]:
filas_nulas = gynecological[gynecological.columns.drop('ehr')].isnull().sum(axis=1) >=4
len(filas_nulas[filas_nulas].index.tolist())

30

In [775]:
gynecological.drop(filas_nulas[filas_nulas].index.tolist(), inplace=True)

In [776]:
len(gynecological.ehr.unique())


7656

In [777]:
elementos_comunes = set(gynecological['ehr']).intersection(set(patients['ehr']))
len(elementos_comunes)

7280

In [778]:
gynecological_patients = pd.merge(gynecological, patients, on='ehr')

In [779]:
gynecological_patients

,ehr,pregnancy,birth,caesarean,abort,menarche_age,er,her2,ki67,pr,birth_date,diagnosis_date,dead
0,10011773,-8.0,0.0,NaN,0.0,NaN,0.0,0.0,19.0,NaN,1959-07-05,2015-04-02,0
1,10030299,3.0,NaN,0.0,0.0,NaN,1.0,NaN,18.0,1.0,1966-08-16,2019-06-27,0
2,10030824,0.0,0.0,0.0,NaN,NaN,1.0,0.0,NaN,1.0,1953-03-03,2018-09-07,0
3,10053435,2.0,NaN,0.0,1.0,21.0,1.0,0.0,NaN,1.0,1957-08-05,2017-12-01,0
4,10111454,0.0,-6.0,NaN,0.0,16.0,1.0,0.0,72.0,1.0,1961-04-02,2017-06-15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7275,99948591,9.0,2.0,-4.0,0.0,12.0,1.0,0.0,12.0,1.0,1955-01-08,2022-02-26,0
7276,99961100,2.0,2.0,0.0,0.0,19.0,1.0,0.0,21.0,1.0,1970-09-24,2018-01-04,0
7277,99981192,3.0,3.0,0.0,-5.0,11.0,1.0,0.0,14.0,0.0,1949-04-10,2017-10-28,0
7278,99992350,4.0,3.0,NaN,-6.0,11.0,0.0,0.0,16.0,0.0,1939-09-04,2018-12-17,0


## Tumor

In [780]:
tumor

,Unnamed: 0,ehr,n_tumor,t_category,n_category,m_category,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_diagnosis,stage_after_neo,grade,ductal,lobular,neoadjuvant
0,0,10011773,1,IS,0,0,NaN,NaN,NaN,0,NaN,1,NaN,NaN,no
1,1,10020495,1,1,0,0,NaN,NaN,NaN,IA,NaN,2,1.0,NaN,no
2,2,10020495,2,3,1,0,2,0.0,NaN,IA,NaN,2,1.0,NaN,yes
3,3,10030299,1,1,0,0,NaN,NaN,NaN,IA,NaN,1,1.0,NaN,no
4,4,10030824,1,2,1,0,2,2.0,0.0,IIIA,IIIA,3,NaN,1.0,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11162,11162,99966287,1,1,1,0,1,0.0,0.0,IB,IIA,3,1.0,NaN,yes
11163,11163,99981192,1,1,0,0,NaN,NaN,NaN,IA,NaN,2,NaN,NaN,no
11164,11164,99988958,1,0,2,0,1,1.0,0.0,IIIA,IIA,1,1.0,NaN,yes
11165,11165,99992350,1,2,0,0,3,2.0,0.0,IIA,IIIA,2,1.0,NaN,yes


In [781]:
tumor.drop(columns=['Unnamed: 0'], inplace=True)

hay ids que se repiten. una persona puede tener varios tumores

In [782]:
len(tumor.ehr.unique())

9971

In [783]:
elementos_comunes = set(tumor['ehr']).intersection(set(gynecological_patients['ehr']))
len(elementos_comunes)

7262

Tiene mas elementos en comun con 'patients' que con 'gynecological_patients'

In [784]:
elementos_comunes = set(tumor['ehr']).intersection(set(patients['ehr']))
len(elementos_comunes)

9485

In [785]:
tumor_patients = pd.merge(tumor, patients, on='ehr')

Finalmente obtenemos un df en el que se incluye la informacion de todos los pasados salvo 'gynecological'. Sin embargo en este df no hay una unica fila por cada paciente, sino que puede haber varias debido a que un paciente puede tener varios tumores, dando lugar a una duplicidad parcial de filas

In [786]:
tumor_patients

,ehr,n_tumor,t_category,n_category,m_category,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_diagnosis,stage_after_neo,...,ductal,lobular,neoadjuvant,er,her2,ki67,pr,birth_date,diagnosis_date,dead
0,10011773,1,IS,0,0,NaN,NaN,NaN,0,NaN,...,NaN,NaN,no,0.0,0.0,19.0,NaN,1959-07-05,2015-04-02,0
1,10020495,1,1,0,0,NaN,NaN,NaN,IA,NaN,...,1.0,NaN,no,1.0,0.0,9.0,0.0,1953-10-02,2017-12-04,0
2,10020495,2,3,1,0,2,0.0,NaN,IA,NaN,...,1.0,NaN,yes,1.0,0.0,9.0,0.0,1953-10-02,2017-12-04,0
3,10030299,1,1,0,0,NaN,NaN,NaN,IA,NaN,...,1.0,NaN,no,1.0,NaN,18.0,1.0,1966-08-16,2019-06-27,0
4,10030824,1,2,1,0,2,2.0,0.0,IIIA,IIIA,...,NaN,1.0,yes,1.0,0.0,NaN,1.0,1953-03-03,2018-09-07,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10621,99963879,1,2,3,0,3,1.0,0.0,IIIC,0,...,1.0,NaN,yes,1.0,0.0,18.0,1.0,1959-10-13,2021-07-20,0
10622,99966287,1,1,1,0,1,0.0,0.0,IB,IIA,...,1.0,NaN,yes,1.0,0.0,NaN,0.0,1978-12-16,2018-08-02,0
10623,99981192,1,1,0,0,NaN,NaN,NaN,IA,NaN,...,NaN,NaN,no,1.0,0.0,14.0,0.0,1949-04-10,2017-10-28,0
10624,99992350,1,2,0,0,3,2.0,0.0,IIA,IIIA,...,1.0,NaN,yes,0.0,0.0,16.0,0.0,1939-09-04,2018-12-17,0


In [787]:
len(tumor_patients.ehr.unique())

9485

Creamos tambien un df mas reducido pero que sí disponga de los datos de 'gynecological'

In [788]:
tumor_patients_gyn = pd.merge(tumor, gynecological_patients, on='ehr')

In [789]:
tumor_patients_gyn

,ehr,n_tumor,t_category,n_category,m_category,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_diagnosis,stage_after_neo,...,caesarean,abort,menarche_age,er,her2,ki67,pr,birth_date,diagnosis_date,dead
0,10011773,1,IS,0,0,NaN,NaN,NaN,0,NaN,...,NaN,0.0,NaN,0.0,0.0,19.0,NaN,1959-07-05,2015-04-02,0
1,10030299,1,1,0,0,NaN,NaN,NaN,IA,NaN,...,0.0,0.0,NaN,1.0,NaN,18.0,1.0,1966-08-16,2019-06-27,0
2,10030824,1,2,1,0,2,2.0,0.0,IIIA,IIIA,...,0.0,NaN,NaN,1.0,0.0,NaN,1.0,1953-03-03,2018-09-07,0
3,10053435,1,0,0,0,NaN,NaN,NaN,IV,NaN,...,0.0,1.0,21.0,1.0,0.0,NaN,1.0,1957-08-05,2017-12-01,0
4,10111454,1,1,0,0,1,0.0,0.0,IA,IA,...,NaN,0.0,16.0,1.0,0.0,72.0,1.0,1961-04-02,2017-06-15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8154,99948591,1,2,0,0,NaN,NaN,NaN,IIA,NaN,...,-4.0,0.0,12.0,1.0,0.0,12.0,1.0,1955-01-08,2022-02-26,0
8155,99961100,1,1,2,0,1,0.0,0.0,IIA,IB,...,0.0,0.0,19.0,1.0,0.0,21.0,1.0,1970-09-24,2018-01-04,0
8156,99981192,1,1,0,0,NaN,NaN,NaN,IA,NaN,...,0.0,-5.0,11.0,1.0,0.0,14.0,0.0,1949-04-10,2017-10-28,0
8157,99992350,1,2,0,0,3,2.0,0.0,IIA,IIIA,...,NaN,-6.0,11.0,0.0,0.0,16.0,0.0,1939-09-04,2018-12-17,0


Nuestro objetivo consiste en predecir si en un paciente habra o no una recaida. No contamos con una variable directamente que nos indique si un paciente recayó o no, pero haciendo uso de este dataset podemos crear una variable asi. 

Un paciente ha recaido si ha tenido mas de un tumor, por lo que podemos construir un nuevo dataframe con una columna binaria 'recaida' si existen 2 filas con mismo 'ehr', ya que cada una correspondera a un 'n_tumor' distinto. Para predecir si un paciente recaera o no tan solo necesitamos la informacion del primer tumor, por tanto eliminaremos las filas que hacen referencia a los tumores reincidentes y de esta forma habra una unica fila por paciente.

In [790]:
tumor_patients_recaida = tumor_patients.copy()
tumor_patients_recaida['recaida'] = tumor_patients_recaida.duplicated(subset='ehr', keep='first').astype(int)
tumor_patients_recaida.drop_duplicates(subset='ehr', keep='first', inplace=True)
tumor_patients_recaida.drop(columns=['n_tumor'], inplace=True)


In [791]:
tumor_patients_recaida

,ehr,t_category,n_category,m_category,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_diagnosis,stage_after_neo,grade,...,lobular,neoadjuvant,er,her2,ki67,pr,birth_date,diagnosis_date,dead,recaida
0,10011773,IS,0,0,NaN,NaN,NaN,0,NaN,1,...,NaN,no,0.0,0.0,19.0,NaN,1959-07-05,2015-04-02,0,0
1,10020495,1,0,0,NaN,NaN,NaN,IA,NaN,2,...,NaN,no,1.0,0.0,9.0,0.0,1953-10-02,2017-12-04,0,0
3,10030299,1,0,0,NaN,NaN,NaN,IA,NaN,1,...,NaN,no,1.0,NaN,18.0,1.0,1966-08-16,2019-06-27,0,0
4,10030824,2,1,0,2,2.0,0.0,IIIA,IIIA,3,...,1.0,yes,1.0,0.0,NaN,1.0,1953-03-03,2018-09-07,0,0
5,10041592,1,1,0,4,0.0,0.0,IB,IA,2,...,NaN,yes,0.0,0.0,65.0,0.0,1959-07-06,2018-11-19,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10621,99963879,2,3,0,3,1.0,0.0,IIIC,0,2,...,NaN,yes,1.0,0.0,18.0,1.0,1959-10-13,2021-07-20,0,0
10622,99966287,1,1,0,1,0.0,0.0,IB,IIA,3,...,NaN,yes,1.0,0.0,NaN,0.0,1978-12-16,2018-08-02,0,0
10623,99981192,1,0,0,NaN,NaN,NaN,IA,NaN,2,...,NaN,no,1.0,0.0,14.0,0.0,1949-04-10,2017-10-28,0,0
10624,99992350,2,0,0,3,2.0,0.0,IIA,IIIA,2,...,NaN,yes,0.0,0.0,16.0,0.0,1939-09-04,2018-12-17,0,0


In [792]:
tumor_patients_recaida.describe()


,ehr,n_category,m_category,n_category_after_neoadj,m_category_after_neoadj,grade,ductal,lobular,er,her2,ki67,pr,dead,recaida
count,9.485000e+03,9485.000000,9485.000000,5171.000000,5171.000000,9485.000000,5383.0,1312.0,8786.000000,8729.000000,8786.000000,8761.000000,9485.000000,9485.0
mean,5.481923e+07,0.847127,0.098577,0.785728,0.099401,1.939062,1.0,1.0,0.664352,0.158667,21.275780,0.666705,0.112599,0.0
std,2.604873e+07,0.988671,0.298109,0.970013,0.299228,0.697254,0.0,0.0,0.472243,0.365386,22.252437,0.471418,0.316118,0.0
min,1.001177e+07,0.000000,0.000000,0.000000,0.000000,1.000000,1.0,1.0,0.000000,0.000000,-57.000000,0.000000,0.000000,0.0
25%,3.208911e+07,0.000000,0.000000,0.000000,0.000000,1.000000,1.0,1.0,0.000000,0.000000,12.000000,0.000000,0.000000,0.0
50%,5.464934e+07,1.000000,0.000000,0.000000,0.000000,2.000000,1.0,1.0,1.000000,0.000000,18.000000,1.000000,0.000000,0.0
75%,7.739152e+07,1.000000,0.000000,1.000000,0.000000,2.000000,1.0,1.0,1.000000,0.000000,28.000000,1.000000,0.000000,0.0
max,9.999442e+07,3.000000,1.000000,3.000000,1.000000,3.000000,1.0,1.0,1.000000,1.000000,137.000000,1.000000,1.000000,0.0


Como podemos ver hay una variable 'lobular' para la cual faltan el 86% de los datos, por lo que en primera instancia optamos por prescindirs de esta variable. Tambien faltan los valores de 't_category_after_neoadj', 'n_category_after_neoadj', 'm_category_after_neoadj' y 'stage_after_neo' para el 45.4% de los datos. Esto es debido a que estas variables solo toman valores en pacientes a los que se les aplico un tratamiento neoadyuvante, por lo que los que no lo recibieron tienen nulo en todas estas columnas. 

Sabiendo esto podemos optar por no tener en cuenta estas variables, o crear una nueva variable que indique si el paciente recibio o no el tratamiento neoadyuvante.

In [793]:
tumor_patients_recaida.isnull().sum()

ehr                           0
t_category                    0
n_category                    0
m_category                    0
t_category_after_neoadj    4314
n_category_after_neoadj    4314
m_category_after_neoadj    4314
stage_diagnosis               0
stage_after_neo            4314
grade                         0
ductal                     4102
lobular                    8173
neoadjuvant                   0
er                          699
her2                        756
ki67                        699
pr                          724
birth_date                    0
diagnosis_date                0
dead                          0
recaida                       0
dtype: int64

En un primer momento parecia que faltaban una gran cantidad de datos para la variable binaria 'lobular', sin embargo si nos fijamos en sus valores todo ellos corresponden al valor '1', por lo que parece que los nulos no lo son realmente, sino que corresponderian a la clase '0'

In [794]:
print(tumor_patients_recaida.lobular.unique())
tumor_patients_recaida['lobular'] = tumor_patients_recaida['lobular'].notnull().astype(int)

[nan  1.]


En cuanto a las variables 't_category_after_neoadj', 'n_category_after_neoadj', 'm_category_after_neoadj' en primer lugar optamos por no considerarlas.

In [795]:
tumor_patients_recaida[tumor_patients_recaida[['t_category_after_neoadj', 'n_category_after_neoadj', 'm_category_after_neoadj', 'stage_after_neo']].isnull().all(axis=1)]


,ehr,t_category,n_category,m_category,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_diagnosis,stage_after_neo,grade,...,lobular,neoadjuvant,er,her2,ki67,pr,birth_date,diagnosis_date,dead,recaida
0,10011773,IS,0,0,NaN,NaN,NaN,0,NaN,1,...,0,no,0.0,0.0,19.0,NaN,1959-07-05,2015-04-02,0,0
1,10020495,1,0,0,NaN,NaN,NaN,IA,NaN,2,...,0,no,1.0,0.0,9.0,0.0,1953-10-02,2017-12-04,0,0
3,10030299,1,0,0,NaN,NaN,NaN,IA,NaN,1,...,0,no,1.0,NaN,18.0,1.0,1966-08-16,2019-06-27,0,0
6,10053435,0,0,0,NaN,NaN,NaN,IV,NaN,3,...,0,no,1.0,0.0,NaN,1.0,1957-08-05,2017-12-01,0,0
12,10115313,1,0,1,NaN,NaN,NaN,IA,NaN,3,...,0,no,1.0,0.0,14.0,0.0,1957-09-05,2019-06-11,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10613,99859482,0,3,0,NaN,NaN,NaN,IV,NaN,2,...,0,no,NaN,0.0,19.0,1.0,1964-10-06,2022-03-28,0,0
10617,99915606,IS,0,0,NaN,NaN,NaN,IIB,NaN,2,...,0,no,0.0,0.0,49.0,1.0,1961-03-30,2017-04-28,0,0
10618,99948515,1,0,0,NaN,NaN,NaN,IA,NaN,3,...,1,no,0.0,0.0,25.0,1.0,1954-06-19,2018-04-24,0,0
10619,99948591,2,0,0,NaN,NaN,NaN,IIA,NaN,2,...,1,no,1.0,0.0,12.0,1.0,1955-01-08,2022-02-26,0,0


Procedemos a eliminar las columnas discutidas.

In [796]:
tumor_patients_recaida.drop(columns=['t_category_after_neoadj', 'n_category_after_neoadj', 'm_category_after_neoadj','stage_after_neo', 'lobular'], inplace=True)


In [797]:
tumor_patients_recaida

,ehr,t_category,n_category,m_category,stage_diagnosis,grade,ductal,neoadjuvant,er,her2,ki67,pr,birth_date,diagnosis_date,dead,recaida
0,10011773,IS,0,0,0,1,NaN,no,0.0,0.0,19.0,NaN,1959-07-05,2015-04-02,0,0
1,10020495,1,0,0,IA,2,1.0,no,1.0,0.0,9.0,0.0,1953-10-02,2017-12-04,0,0
3,10030299,1,0,0,IA,1,1.0,no,1.0,NaN,18.0,1.0,1966-08-16,2019-06-27,0,0
4,10030824,2,1,0,IIIA,3,NaN,yes,1.0,0.0,NaN,1.0,1953-03-03,2018-09-07,0,0
5,10041592,1,1,0,IB,2,1.0,yes,0.0,0.0,65.0,0.0,1959-07-06,2018-11-19,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10621,99963879,2,3,0,IIIC,2,1.0,yes,1.0,0.0,18.0,1.0,1959-10-13,2021-07-20,0,0
10622,99966287,1,1,0,IB,3,1.0,yes,1.0,0.0,NaN,0.0,1978-12-16,2018-08-02,0,0
10623,99981192,1,0,0,IA,2,NaN,no,1.0,0.0,14.0,0.0,1949-04-10,2017-10-28,0,0
10624,99992350,2,0,0,IIA,2,1.0,yes,0.0,0.0,16.0,0.0,1939-09-04,2018-12-17,0,0


In [798]:
tumor_patients_recaida.isnull().sum()

ehr                   0
t_category            0
n_category            0
m_category            0
stage_diagnosis       0
grade                 0
ductal             4102
neoadjuvant           0
er                  699
her2                756
ki67                699
pr                  724
birth_date            0
diagnosis_date        0
dead                  0
recaida               0
dtype: int64

In [799]:
tumor_patients_recaida.ductal.unique()

array([nan,  1.])